In [2]:
from collections import defaultdict as dd
import os
import json
import sqlite3
import ast
import re
from sqlite3 import Error
from collections import Counter

def read_json(file):
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    return json_data

def read_jsonl(file):
    with open(file) as json_file:
        json_data = [json.loads(line) for line in json_file]
    
    return json_data

In [5]:
bs="6"
qv_link="false"
dist_relation="true"
orthog="false"
orth_init="false"
bi_way="true"
bi_match="true"

exp = f'bs={bs},qv_link={qv_link},dist={dist_relation},orthog={orthog},orth_init={orth_init},bi_way={bi_way},bi_match={bi_match}'
ratsql_infereval_loc = f'logdir/bert_run/{exp}/ie_dirs/'
ratsql_train = 'rat-sql/data/spider/train_spider.json'
ratsql_train_others = 'rat-sql/data/spider/train_others.json'
# step = [1] + [(5000 * x) + 100 for x in range(1, 17)] + [81000]
step = 60500
ratsql_infer = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.infer')
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

In [6]:
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

evaluation = read_jsonl(ratsql_eval)[0]

In [8]:
per_item = evaluation['per_item']

In [10]:
for p in per_item:
    exact = p['exact']
    if exact:
        print(f"predicted: {p['predicted']}")
        print(f"gold: {p['gold']}")

predicted: SELECT Count(*) FROM singer
gold: SELECT count(*) FROM singer
predicted: SELECT Count(*) FROM singer
gold: SELECT count(*) FROM singer
predicted: SELECT singer.Name, singer.Country, singer.Age FROM singer ORDER BY singer.Age Desc
gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC
predicted: SELECT singer.Name, singer.Country, singer.Age FROM singer ORDER BY singer.Age Desc
gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC
predicted: SELECT Avg(singer.Age), Min(singer.Age), Max(singer.Age) FROM singer WHERE singer.Country = 'terminal'
gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
predicted: SELECT singer.Song_Name, singer.Song_release_year FROM singer ORDER BY singer.Age Asc LIMIT 1
gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1
predicted: SELECT singer.Song_Name, singer.Song_release_year FROM singer ORDER BY singer.Age Asc LIMIT 1
gold: SELECT song_name ,  song_release_year FR

In [29]:
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

evaluation = read_jsonl(ratsql_eval)[0]

In [30]:
per_item = evaluation['per_item']

for pi in per_item:
    print(pi['exact'])
    print(pi['predicted'])
    print(pi['gold'])
    print('='*30)

0
SELECT Count(DISTINCT singer.Name) FROM singer
SELECT count(*) FROM singer
0
SELECT Count(DISTINCT singer_in_concert.Singer_ID) FROM singer_in_concert
SELECT count(*) FROM singer
0
SELECT stadium.Name, singer.Country, singer.Age FROM stadium JOIN singer JOIN concert ON stadium.Stadium_ID = concert.Stadium_ID JOIN singer_in_concert ON concert.concert_ID = singer_in_concert.concert_ID AND singer_in_concert.Singer_ID = singer.Singer_ID ORDER BY singer.Age Desc
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
0
SELECT stadium.Name, singer.Country, singer.Age FROM stadium JOIN singer JOIN concert ON stadium.Stadium_ID = concert.Stadium_ID JOIN singer_in_concert ON concert.concert_ID = singer_in_concert.concert_ID AND singer_in_concert.Singer_ID = singer.Singer_ID ORDER BY singer.Age Desc
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
0
SELECT Avg(stadium.Average), Min(singer.Age), Max(singer.Age) FROM stadium JOIN singer JOIN concert ON stadium.Stadium_ID = concert

In [3]:
bs="6"
loss="label_smooth"
qv_link="false"
dist_relation="true"
orthog="false"
orth_init="false"
bi_match="true"
bi_way="true"

'{"att": 1, "bert_lr": 3e-06, "bert_token_type": true, "bert_version": "bert-large-uncased-whole-word-masking", "bs": 6, "clause_order": null, "cv_link": true, "data_path": "rat-sql/data/spider/", "decoder_hidden_size": 512, "end_lr": 0, "end_with_from": true, "loss": "label_smooth", "lr": 0.000744, "max_steps": 121000, "num_batch_accumulated": 4, "num_layers": 8, "qv_link": '+qv_link+ ', "dist_relation": ' + dist_relation + ', "use_orthogonal": ' + orthog + ', "use_orth_init": '+orth_init+', "sc_link": true, "summarize_header": "avg", "use_align_loss": true, "use_align_mat": true, "use_column_type": false, "bi_match": '+bi_match+', "bi_way": '+bi_way+'}'

'{"att": 1, "bert_lr": 3e-06, "bert_token_type": true, "bert_version": "bert-large-uncased-whole-word-masking", "bs": 6, "clause_order": null, "cv_link": true, "data_path": "rat-sql/data/spider/", "decoder_hidden_size": 512, "end_lr": 0, "end_with_from": true, "loss": "label_smooth", "lr": 0.000744, "max_steps": 121000, "num_batch_accumulated": 4, "num_layers": 8, "qv_link": false, "dist_relation": true, "use_orthogonal": false, "use_orth_init": false, "sc_link": true, "summarize_header": "avg", "use_align_loss": true, "use_align_mat": true, "use_column_type": false, "bi_match": true, "bi_way": true}'

In [28]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from pyvis.network import Network
from sklearn.metrics.pairwise import cosine_similarity

import itertools
from collections import defaultdict

import scipy
import sys
sys.path.append('rat-sql')
sys.path.append('../')

import attr
import _jsonnet
import json 

from ratsql.commands.infer import Inferer
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"

os.chdir('/repo')

@attr.s
class InferConfig:
    config = attr.ib()
    config_args = attr.ib()
    logdir = attr.ib()
    section = attr.ib()
    beam_size = attr.ib()
    output = attr.ib()
    step = attr.ib()
    use_heuristic = attr.ib(default=False)
    mode = attr.ib(default="infer")
    limit = attr.ib(default=None)
    output_history = attr.ib(default=False)

bs="6"
qv_link="true"
dist_relation="false"
orthog="false"
orth_init="false"
bi_match="true"
bi_way="true"

infer_config = InferConfig(config='rat-sql/configs/spider/nl2code-bert.jsonnet', 
                            config_args='{"att": 1, "bert_lr": 3e-06, "bert_token_type": true, "bert_version": "bert-large-uncased-whole-word-masking", "bs": 6, "clause_order": null, "cv_link": true, "data_path": "rat-sql/data/spider/", "decoder_hidden_size": 512, "end_lr": 0, "end_with_from": true, "loss": "label_smooth", "lr": 0.000744, "max_steps": 121000, "num_batch_accumulated": 4, "num_layers": 8, "qv_link": '+qv_link+ ', "dist_relation": ' + dist_relation + ', "use_orthogonal": ' + orthog + ', "use_orth_init": '+orth_init+', "sc_link": true, "summarize_header": "avg", "use_align_loss": true, "use_align_mat": true, "use_column_type": false, "bi_match": '+bi_match+', "bi_way": '+bi_way+'}', 
                            logdir='rat-sql/logdir/bert_run', 
                            section='val',
                            beam_size=1, 
                            output='__LOGDIR__/ie_dirs/bert_run_true_1-step1.infer', 
                            step=1, use_heuristic=True, mode='infer', limit=None, output_history=False)

config = json.loads(_jsonnet.evaluate_file(infer_config.config, tla_codes={'args': infer_config.config_args}))

inferer = Inferer(config)

logdir = os.path.join(infer_config.logdir, config['model_name'])
step = 10500

model = inferer.load_model(logdir, step)

relation_k_emb_0 = model.state_dict()['encoder.encs_update.encoder.layers.0.relation_k_emb.weight'].cpu()
relation_k_emb_1 = model.state_dict()['encoder.encs_update.encoder.layers.1.relation_k_emb.weight'].cpu()
relation_k_emb_2 = model.state_dict()['encoder.encs_update.encoder.layers.2.relation_k_emb.weight'].cpu()
relation_k_emb_3 = model.state_dict()['encoder.encs_update.encoder.layers.3.relation_k_emb.weight'].cpu()
relation_k_emb_4 = model.state_dict()['encoder.encs_update.encoder.layers.4.relation_k_emb.weight'].cpu()
relation_k_emb_5 = model.state_dict()['encoder.encs_update.encoder.layers.5.relation_k_emb.weight'].cpu()
relation_k_emb_6 = model.state_dict()['encoder.encs_update.encoder.layers.6.relation_k_emb.weight'].cpu()
relation_k_emb_7 = model.state_dict()['encoder.encs_update.encoder.layers.7.relation_k_emb.weight'].cpu()

relation_ks = [relation_k_emb_0, relation_k_emb_1, relation_k_emb_2, relation_k_emb_3, relation_k_emb_4, relation_k_emb_5, relation_k_emb_6, relation_k_emb_7]

relation_v_emb_0 = model.state_dict()['encoder.encs_update.encoder.layers.0.relation_v_emb.weight'].cpu()
relation_v_emb_1 = model.state_dict()['encoder.encs_update.encoder.layers.1.relation_v_emb.weight'].cpu()
relation_v_emb_2 = model.state_dict()['encoder.encs_update.encoder.layers.2.relation_v_emb.weight'].cpu()
relation_v_emb_3 = model.state_dict()['encoder.encs_update.encoder.layers.3.relation_v_emb.weight'].cpu()
relation_v_emb_4 = model.state_dict()['encoder.encs_update.encoder.layers.4.relation_v_emb.weight'].cpu()
relation_v_emb_5 = model.state_dict()['encoder.encs_update.encoder.layers.5.relation_v_emb.weight'].cpu()
relation_v_emb_6 = model.state_dict()['encoder.encs_update.encoder.layers.6.relation_v_emb.weight'].cpu()
relation_v_emb_7 = model.state_dict()['encoder.encs_update.encoder.layers.7.relation_v_emb.weight'].cpu()

relation_vs = [relation_v_emb_0, relation_v_emb_1, relation_v_emb_2, relation_v_emb_3, relation_v_emb_4, relation_v_emb_5, relation_v_emb_6, relation_v_emb_7]

relation_index = {('qq_dist', -2): 0,
                    ('qq_dist', -1): 1,
                    ('qq_dist', 0): 2,
                    ('qq_dist', 1): 3,
                    ('qq_dist', 2): 4,
                    'qc_default': 5,
                    'qt_default': 6,
                    'cq_default': 7,
                    'cc_default': 8,
                    'cc_foreign_key_forward': 9,
                    'cc_foreign_key_backward': 10,
                    'cc_table_match': 11,
                    ('cc_dist', -2): 12,
                    ('cc_dist', -1): 13,
                    ('cc_dist', 0): 14,
                    ('cc_dist', 1): 15,
                    ('cc_dist', 2): 16,
                    'ct_default': 17,
                    'ct_foreign_key': 18,
                    'ct_primary_key': 19,
                    'ct_table_match': 20,
                    'ct_any_table': 21,
                    'tq_default': 22,
                    'tc_default': 23,
                    'tc_primary_key': 24,
                    'tc_table_match': 25,
                    'tc_any_table': 26,
                    'tc_foreign_key': 27,
                    'tt_default': 28,
                    'tt_foreign_key_forward': 29,
                    'tt_foreign_key_backward': 30,
                    'tt_foreign_key_both': 31,
                    ('tt_dist', -2): 32,
                    ('tt_dist', -1): 33,
                    ('tt_dist', 0): 34,
                    ('tt_dist', 1): 35,
                    ('tt_dist', 2): 36,
                    'qcCEM': 37,
                    'cqCEM': 38,
                    'qtTEM': 39,
                    'tqTEM': 40,
                    'qcCPM': 41,
                    'cqCPM': 42,
                    'qtTPM': 43,
                    'tqTPM': 44,
                    'qcNUMBER': 45,
                    'cqNUMBER': 46,
                    'qcTIME': 47,
                    'cqTIME': 48,
                    'qcCELLMATCH': 49,
                    'cqCELLMATCH': 50}

relation_index_list = list(relation_index.keys())

print(f'bs: {bs}, qv_link: {qv_link}, dist_relation: {dist_relation}, orthog: {orthog}, orth_init: {orth_init}, bi_match: {bi_match}, bi_way: {bi_way}')

WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bert_version': 'bert-large-uncased-whole-word-masking', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'rat-sql/data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink'}}
WARNING <class 'ratsql.models.spider.spider_enc.SpiderEncoderBert'>: superfluous {'enc_qv_link': True, 'qv_link': True, 'use_orthogonal': False}


Used Relations: 46


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from rat-sql/logdir/bert_run/bs=6,qv_link=true,dist=false,orthog=false,orth_init=false,bi_way=true,bi_match=true/model_checkpoint-00010500
bs: 6, qv_link: true, dist_relation: false, orthog: false, orth_init: false, bi_match: true, bi_way: true


In [27]:
model.encoder.encs_update.relation_ids

{'qq_default': 0,
 'qc_default': 1,
 'qt_default': 2,
 'cc_default': 3,
 'cc_foreign_key_forward': 4,
 'cc_table_match': 5,
 'ct_default': 6,
 'ct_foreign_key': 7,
 'ct_primary_key': 8,
 'ct_table_match': 9,
 'ct_any_table': 10,
 'tt_default': 11,
 'tt_foreign_key_forward': 12,
 'qcCEM': 13,
 'qtTEM': 14,
 'qcNUMBER': 15,
 'qcTIME': 16,
 'qcCELLMATCH': 17}

In [ ]:
read_jsonl()